<a href="https://colab.research.google.com/github/shreyas1209/Emotion-Detection/blob/main/Emotion_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#testing data preprocessing
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import PIL
import os
import matplotlib.pyplot as plt
import matplotlib.image as img
from torchvision import datasets
from torchvision import transforms
from PIL import Image, ImageDraw
from torch.utils.data import Dataset, DataLoader, TensorDataset
from numpy import genfromtxt
import torchvision.utils as vutils
import cv2

In [ ]:

#mounting google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

Using pandas to see how the train and validation data is stored in the csv file


In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Emotion Detector/emotion_detection/icml_face_data.csv')
df


In [ ]:
emotion_dict = {0 :'Angry' ,1 :'Disgusted' , 2 :'Fear' ,3 : 'Happy' ,4 :'Sad' ,5 :'Surprise' ,6 : 'Neutral' }

Exploratory Data Analysis

In [ ]:
train_num_counts = np.zeros(7)
val_num_counts = np.zeros(7)
for i in range(7):
  train_num_counts[i] = df['emotion'][(df['emotion']==i) &(df[' Usage'] == 'Training')].value_counts()
  val_num_counts[i] = df['emotion'][(df['emotion']==i) &(df[' Usage'] == 'PrivateTest')].value_counts()
print(train_num_counts,val_num_counts)

Plotting the bar graph for training data

In [ ]:

plt.bar(emotion_dict.values(),train_num_counts,width = 0.5)
plt.xlabel('Emotion',fontdict={'size': 16})
plt.ylabel('No of instances',fontdict={'size': 16})
plt.title("Training Data",fontdict={'size': 16})
plt.grid()
plt.show()


The number of data items belonging to the class 'Disgusted' is quite low compared to the other classes

Plotting the graph for validation data

In [ ]:
plt.bar(emotion_dict.values(),val_num_counts,width = 0.5)
plt.xlabel('Emotion',fontdict={'size': 16})
plt.ylabel('No of instances',fontdict={'size': 16})
plt.title("Validation Data",fontdict={'size': 16})
plt.grid()

plt.show()

function that converts string data to tensor

In [ ]:
def totensor(dataval):
  data = dataval.split()
  data = [int(i) for i in data]
  data = torch.tensor(data, dtype = torch.float)
  return data




    
    


In [ ]:
def DATASET (df,normalize = 'False'):
  data = df[' pixels']
  data = [totensor(i) for i in data]
  data = torch.stack([image for image in data])
  data = (data.reshape(data.shape[0],1,48,48)/256)
  if normalize == 'True':
    datastack = torch.stack([data[i] for i in range(len(data))],dim= 2)
    mean = datastack.view(1,-1).mean(dim = 1)
    std = datastack.view(1,-1).std(dim = 1)
    print(mean, std)
    data = ((data-mean)/std)
    

  labels = torch.tensor(df['emotion'].to_numpy())
  dataset = TensorDataset(data, labels)
  return dataset










In [ ]:
train_dataset = DATASET(df[df[' Usage']=='Training'],normalize = 'True')
val_dataset = DATASET(df[df[' Usage']=='PrivateTest'],normalize = 'True')
test_dataset = DATASET(df[df[' Usage']=='PublicTest'],normalize = 'True')

In [ ]:
print(train_dataset[0][0])
plt.imshow(train_dataset[0][0].reshape(48,48))


In [ ]:
val_dataset[0][0]
plt.imshow(val_dataset[0][0].reshape(48,48))

In [ ]:
print(test_dataset[0][0])
plt.imshow(test_dataset[0][0].reshape(48,48))

In [ ]:
batch_size = 128

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size = batch_size)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size)


Importing the training and models module

In [ ]:
import sys

sys.path.append('/content/gdrive/MyDrive/Emotion Detector') 


In [ ]:
from Emotion_train import *
from Emotion_models import *

In [ ]:
model = emonet(7)
learning_rate = 1e-3
epochs = 50
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
loss_func = nn.CrossEntropyLoss()




    
    

    

In [ ]:
start = time.process_time()
model,history = train(train_dataloader,val_dataloader,epochs,model,loss_func,optimizer)
end = time.process_time()
elapsed = end-start
print("Total time :", elapsed)

Post training analysis

In [ ]:
epoch_no = [(i+1) for i in range(epochs)]

plt.plot(epoch_no,history["train_accuracy"], label = 'Train Accuracies')
plt.plot(epoch_no,history["val_accuracy"], label = 'Validation Accuracies')
plt.xlabel("Epochs")
plt.ylabel("Accuracies")
plt.title('Accuracy')
plt.legend()
plt.show()





In [ ]:
plt.plot(epoch_no,history["loss"])
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title('Loss')
plt.show()


In [ ]:
plt.plot(epoch_no,history["lr"])
plt.xlabel("Epochs")
plt.ylabel("Learning Rates")
plt.title('Learnrng rates')
plt.show()


In [1]:
test_score = check_accuracy(test_dataloader,model)
print("Test dataset score :",test_score)


NameError: ignored